<h2>Desafio 2 - Machine Learning


<h3>Grupo 7</h3>
<ul>
    <li>Ignacio Mendieta</li>
    <li>Laura Jazmín Chao</li>
    <li>Juan Nicolás Capistrano</li>
    <li>Betiana Srur</li>
    <li>Marecelo Carrizo</li>
    
</ul>
    

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [6]:
import numpy as np
import pandas as pd
import re

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Visualización
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import NumeralTickFormatter # Para imprimir los %
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, Vendors 
from bokeh.resources import INLINE

# Libreria para analisis ML
from sklearn import datasets, linear_model

pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

In [7]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
10231,10231,sell,house,Las Heras,|Argentina|Mendoza|Las Heras|,Argentina,Mendoza,3848352.0,"-32.839753,-68.875265",-32.839753,-68.875265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,http://www.properati.com.ar/166tl_venta_casa_l...,Corredor Responsable: Real Estate New Generati...,"CASA EN EL CHALLAO, LAS HERAS, MENDOZA",https://thumbs4.properati.com/9/DAOYHDA_gCoihp...
80565,80565,sell,house,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.7534023,-58.4086388",-34.753402,-58.408639,130000.0,USD,2293785.0,130000.0,140.0,140.0,928.571429,928.571429,NaN,NaN,NaN,http://www.properati.com.ar/1awzl_venta_casa_l...,"CHALET EN DOS PLANTAS , HALL RECIBIDOR ,LIVING...",Casa en Lomas de Zamora Oeste,https://thumbs4.properati.com/8/WKCbYUaZmbCGr2...
25884,25884,sell,apartment,Boedo,|Argentina|Capital Federal|Boedo|,Argentina,Capital Federal,3436003.0,NaN,NaN,NaN,639800.0,USD,11288951.1,639800.0,30.0,30.0,21326.666667,21326.666667,NaN,1.0,NaN,http://www.properati.com.ar/17qhy_venta_depart...,bf226- COMPRA UN DEPTO EN BOEDO A ESTRENAR Y E...,bf226- COMPRA UN DEPTO EN BOEDO A ESTRENAR Y E...,https://thumbs4.properati.com/2/zd_b2bw5W3Z69O...
108917,108917,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,145000.0,USD,2558452.5,145000.0,78.0,78.0,1858.974359,1858.974359,NaN,NaN,NaN,http://www.properati.com.ar/1c1k8_venta_casa_r...,"Casa al frente, desarrollada en planta baja, c...",Barrio Sarmiento - Perez Galdos 600,https://thumbs4.properati.com/3/9QAnpWh4aVgiun...
75971,75971,sell,house,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.7538075,-58.4143123",-34.753808,-58.414312,175000.0,USD,3087787.5,175000.0,NaN,75.0,NaN,2333.333333,NaN,NaN,NaN,http://www.properati.com.ar/1aqza_venta_casa_l...,"LOTE DE 8.60X 18.88Casa en PH al frente, sin e...",Casa - Lomas de Zamora Oeste,https://thumbs4.properati.com/2/SwCFaM3RZDAI-a...


In [8]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Selección de los datos</h2>

En base a los análisis previos realizados sobre el Dataset, tomaremos los datos de CABA únicamente para realizar un modelo de machine learning. 

In [9]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data.state_name == 'Capital Federal' 
data_caba = data.loc[data_caba_mask, :]

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...


In [10]:
data_caba.shape

(32316, 26)

In [11]:
data_caba.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

<a id="section_dataset_nulos"></a> 
<h3>Cálculo de cantidad de nulos</h3>

[volver a TOC](#section_toc)

In [12]:
# # Calculamos cantidad de nulos por campo
# cant_nulos_por_campo = data_caba.isnull().sum()
# cant_nulos_por_campo

In [13]:
# # Calculamos porcentaje de nulos por campo
# cant_registros = data_caba.shape[0]
# print(f'Cantidad Original de registros: {cant_registros}\n')
# #len(data.index) es lo mismo

# porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
# print(porc_nulos_por_campo)

<h4> Distribución de <strong>property_type</strong>

In [14]:
# Contamos cantidad de valores de cada elemento en property_type y operation para ver distribución
property_type_count = data_caba.property_type.value_counts()
print(f'property_type: \n{property_type_count}')

property_type: 
apartment    27037
house         2087
PH            1828
store         1364
Name: property_type, dtype: int64


In [15]:
# # Convertimos el count anterior en las categorías del gráfico
# categories = np.array(property_type_count.index)

# cord_x = data_caba.property_type.value_counts() / data_caba.shape[0]
# palette = ['#440154', '#29788E', '#22A784', '#FDE724']

# p = figure(x_range=categories, plot_width=500, plot_height=200)
# p.vbar(x=categories, top=cord_x, width=0.6,
#        color=palette)

# p.yaxis.formatter = NumeralTickFormatter(format='0 %')
# output_notebook(resources=INLINE)
# show(p)

<h4>Distribución de <strong>currency</strong></h4>

In [16]:
data_caba.currency.value_counts()

USD    27826
ARS     1190
PEN        2
Name: currency, dtype: int64

<h4>Distribución de <strong>barrio</strong></h4>

In [17]:
place_name_count = data_caba.place_name.value_counts()
# place_name_count.sort_index()
place_name_count

Belgrano                2992
Palermo                 2885
Caballito               2273
Villa Urquiza           1629
Recoleta                1547
Flores                  1354
Villa Crespo            1331
Capital Federal         1297
San Telmo               1216
Almagro                 1165
Barrio Norte            1140
Boedo                    890
Nuñez                    778
Balvanera                667
Puerto Madero            647
San Cristobal            614
Saavedra                 542
Monserrat                538
Palermo Hollywood        532
Floresta                 516
Villa del Parque         488
Villa Luro               432
Barracas                 425
Villa Devoto             420
Mataderos                418
Liniers                  408
Palermo Soho             394
Colegiales               352
Parque Patricios         322
Congreso                 304
Coghlan                  259
Retiro                   248
Palermo Chico            232
Chacarita                227
Boca          

<h2>Limpieza de Datos</h2>

<a id="section_dataset_drop"></a> 
<h3> Drop de columnas innecesarias </h3>

[volver a TOC](#section_toc)

In [18]:
# Creamos la lista de columnas a filtrar
drop_columns = ['Unnamed: 0', 'operation', 'place_with_parent_names', 'country_name', 'state_name', 'geonames_id', 'lat-lon','lat', 'lon',
               'expenses','properati_url','title', 'image_thumbnail']

data_caba_clean = data_caba.drop(drop_columns, axis=1)
data_caba_clean.head(5)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,..."
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...
7,apartment,Belgrano,138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...
8,apartment,Belgrano,195000.0,USD,3440677.5,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...



<h3> Limpiamos <strong>currency</strong> </h3>

[volver a TOC](#section_toc)

In [19]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data_caba_clean.currency == 'PEN') | (data_caba_clean.currency == 'UYU')
# Observamos esos registros
data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :]

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
50387,apartment,Capital Federal,380000.0,PEN,2066864.90,117139.33,NaN,NaN,NaN,NaN,NaN,NaN,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...
50388,apartment,Capital Federal,950000.0,PEN,5167162.35,292848.33,NaN,NaN,NaN,NaN,NaN,NaN,Ref.: 56Duplex en venta de 1 dormitorio con ap...


In [20]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data_caba_clean.shape[0]
data_caba_clean.drop(data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data_caba_clean.shape[0]}')

Cantidad de registros original - registro actuales: 2


<h3> Limpiamos <strong>place_name</strong> </h3>

[volver a TOC](#section_toc)

In [21]:
# Dropeamos los de Capital Federal
capi_mask = data_caba_clean.place_name == 'Capital Federal'
data_caba_clean.drop(data_caba_clean.loc[capi_mask, :].index, inplace=True)

# Chequeamos cómo quedaron los datos
data_caba_clean.shape

(31019, 13)

<h3> Volvemos a calcular cantidad de nulos luego de eliminar columnas y registros

In [22]:
# Calculamos cantidad de nulos por campo
cant_nulos_por_campo = data_caba_clean.isnull().sum()
cant_nulos_por_campo

property_type                     0
place_name                        0
price                          2959
currency                       2959
price_aprox_local_currency     2959
price_aprox_usd                2959
surface_total_in_m2            5707
surface_covered_in_m2          1963
price_usd_per_m2               8416
price_per_m2                   4570
floor                         28070
rooms                         15748
description                       0
dtype: int64

In [23]:
# Calculamos porcentaje de nulos por campo
cant_registros = data_caba_clean.shape[0]
print(f'Cantidad Original de registros: {cant_registros}\n')
#len(data.index) es lo mismo

porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
print(porc_nulos_por_campo)

Cantidad Original de registros: 31019

property_type                  0.00
place_name                     0.00
price                          9.54
currency                       9.54
price_aprox_local_currency     9.54
price_aprox_usd                9.54
surface_total_in_m2           18.40
surface_covered_in_m2          6.33
price_usd_per_m2              27.13
price_per_m2                  14.73
floor                         90.49
rooms                         50.77
description                    0.00
dtype: float64


In [24]:
data_caba.shape

(32316, 26)

In [25]:
data_caba.head(20)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
13,13,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0,NaN,http://www.properati.com.ar/15bp9_venta_depart...,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,https://thumbs4.properati.com/2/ZTUjkRzTz6YvcU...
14,14,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0,NaN,http://www.properati.com.ar/15bpa_venta_depart...,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo,https://thumbs4.properati.com/8/Uay2dC732CNlRw...
16,16,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6523561177,-58.5016239381",-34.652356,-58.501624,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0,NaN,http://www.properati.com.ar/15bpj_venta_ph_mat...,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.,https://thumbs4.properati.com/7/XSdJtiE8ZkWgon...
19,19,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.580503566,-58.4058744847",-34.580504,-58.405874,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0,NaN,http://www.properati.com.ar/15bq8_venta_depart...,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",https://thumbs4.properati.com/8/1y9fKHLee-aQQj...
21,21,sell,apartment,Pale

## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATIVAS, Dummy (ejemplo: place_name, property_type, amenities, rooms).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Regresion Lineal Multiple
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



### Primero damos homogenidad a los datos

Vamos a realizar ciertas verificaciones de los datos.

In [26]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos el price_aprox_usd

mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd

display(data_caba_clean.loc[mask_price_validation, :].shape)

# Detectamos que hay 4138 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba_clean.loc[mask_price_validation, :].sample(10))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados. Podemos hacerlo en base al calculo que realizamos en el Desafio 1 ??

(4138, 13)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
76564,apartment,Belgrano,NaN,NaN,NaN,NaN,91.0,81.0,NaN,NaN,NaN,NaN,Departamento en Venta en Belgrano de 2 dormito...
117785,apartment,San Telmo,856240.0,ARS,847307.94,48021.08,22.0,22.0,2182.776364,38920.0,NaN,NaN,Formando Parte del Barrio de San Telmo: Caract...
85838,apartment,Barrio Norte,NaN,NaN,NaN,NaN,42.0,37.0,NaN,NaN,NaN,NaN,FORMA DE PAGO: A CONVENIR BBU2985 AP195266
17842,apartment,Once,NaN,NaN,NaN,NaN,74.0,58.0,NaN,NaN,1.0,3.0,"Departamento luminoso, a metros de la av. Puey..."
66125,apartment,Palermo,NaN,NaN,NaN,NaN,54.0,48.0,NaN,NaN,NaN,NaN,"Unidades de 1, 2 y 3 ambientes. El edificio se..."
24560,apartment,Villa Devoto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VENTA 2 AMBIENTES EN VILLA DEVOTO Departamento...
97316,apartment,Floresta,NaN,NaN,NaN,NaN,73.0,69.0,NaN,NaN,NaN,NaN,Departamento de 3 ambientes con vista a Av. Lo...
118015,apartment,Belgrano,NaN,NaN,NaN,NaN,83.0,73.0,NaN,NaN,NaN,NaN,"3 Ambientes.2 suites, living comedor cocina, t..."
71559,apartment,Palermo Hollywood,NaN,NaN,NaN,NaN,64.0,53.0,NaN,NaN,NaN,NaN,Se implanta sobre un lote de 30 m de frente so...
79931,apartment,Nuñez,NaN,NaN,NaN,NaN,380.0,290.0,NaN,NaN,NaN,NaN,Espectacular departamento en torre Chateau lib...


In [27]:
data_caba_clean.shape

(31019, 13)

<h3> Eliminamos registros según un umbral de datos faltantes </h3>

[volver a TOC](#section_toc)

In [28]:
# Elimino primero los registros donde no tengo nada de informacion. 

umbral = 10
data_caba_clean.dropna(axis = 0, thresh=umbral, inplace=True)
# OJO que utilizando umbral de 10 se fueron casi 10000 datos incompletos. 
#--> se pierden 6000 datos aprox (ver shape de abajo), un 20% de los datos aprox

# Vuelvo a ver como quedaron los datos
mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd
display(data_caba_clean.loc[mask_price_validation, :].shape)

display(data_caba_clean.shape)
display(data_caba_clean.loc[mask_price_validation, :].sample(20))


(1131, 13)

(25627, 13)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
56787,apartment,Nuñez,2618000.0,ARS,2590690.06,146827.06,NaN,44.0,NaN,59500.000000,NaN,1.0,Corredor Responsable: Jorge Salafia - CUCICBA ...
8056,apartment,Constitución,1263800.0,ARS,1250616.57,70878.55,NaN,31.0,NaN,40767.741935,NaN,1.0,* Balcón al FrenteEstá ubicado en una zona est...
91484,apartment,Boedo,4420000.0,ARS,4373892.45,247889.85,114.0,63.0,2174.472368,70158.730159,NaN,NaN,Edificio de categoría PB y 10 Pisos.Piso 10º: ...
85514,apartment,San Telmo,3243275.0,ARS,3209442.44,181894.78,69.0,65.0,2636.156232,49896.538462,NaN,NaN,ASTILLERO se encontrará emplazado en Madero Su...
111135,apartment,Constitución,1353200.0,ARS,1339083.98,75892.43,36.0,32.0,2108.123056,42287.500000,NaN,1.0,Departamento de 1 Ambiente al frente.NO APTO C...
120421,apartment,San Telmo,1755600.0,ARS,1737286.29,98460.50,48.0,44.0,2051.260417,39900.000000,NaN,NaN,"Departamento 48m², Capital Federal, San Telmo,..."
120366,apartment,San Telmo,1319000.0,ARS,1305240.59,73974.36,34.0,31.0,2175.716471,42548.387097,NaN,NaN,"Departamento 34m², Capital Federal, San Telmo,..."
82548,apartment,Palermo,2399325.0,ARS,2374296.14,134562.96,43.0,36.0,3129.371163,66647.916667,NaN,NaN,Exclusivo duplex en 5º y 6º piso con balcón a...
23377,apartment,Caballito,1634000.0,ARS,1616954.68,91640.72,NaN,40.0,NaN,40850.000000,1.0,2.0,Corredor Responsable: Hernan Perrone - C.U.C.I...
117700,apartment,Palermo,1759100.0,ARS,1740749.73,98656.79,37.0,33.0,2666.399730,53306.060606,NaN,NaN,Monoambiente en Avenida Cabildo entre Santos D...


In [24]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan 
#es porque es un departamente y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba

data_caba_clean.surface_total_in_m2.fillna(data_caba_clean.surface_covered_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
28145,apartment,San Cristobal,81900.0,USD,1445084.55,81900.00,40.0,38.0,2047.500000,2155.263158,NaN,1.0,PRONTA ENTREGA!!!!! IMPECABLE MONOAMBIENTE AL...
48419,apartment,Caballito,68000.0,USD,1199826.00,68000.00,35.0,35.0,1942.857143,1942.857143,NaN,2.0,Venta departamento 2 ambientes en Caballito (...
86770,apartment,Villa Crespo,90000.0,USD,1588005.00,90000.00,39.0,34.0,2307.692308,2647.058824,NaN,NaN,VENTA EN POZO - UNIDADES DE 1 Y 2 AMBIENTESDep...
53581,apartment,Belgrano,260000.0,USD,4587570.00,260000.00,76.0,76.0,NaN,3421.052632,NaN,3.0,"Venta - Departamento - Argentina, Buenos Aires..."
109649,PH,San Telmo,198000.0,USD,3493611.00,198000.00,146.0,126.0,1356.164384,1571.428571,NaN,6.0,"DEFENSA 1035, PH DOS PLANTAS8 AMB CON PATIO Y ..."
116162,apartment,Villa Crespo,88000.0,USD,1552716.00,88000.00,39.0,39.0,NaN,2256.410256,4.0,2.0,Corredor Responsable: Graciela A. Destal - CUI...
9010,apartment,Villa Urquiza,1962121.0,ARS,1941653.00,110042.96,42.0,42.0,NaN,46717.166667,NaN,1.0,Edificio en Construcción Fecha de Finalización...
20942,apartment,Barrio Norte,300000.0,USD,5293350.00,300000.00,130.0,93.0,2307.692308,3225.806452,NaN,3.0,DEPARTAMENTO CON BCON TZA SUP..LUMINOSO .A ...
48865,apartment,Barrio Norte,79000.0,USD,1393915.50,79000.00,30.0,30.0,2633.333333,2633.333333,NaN,1.0,"Mono ambiente en excelente zona de facultades,..."
85354,PH,Balvanera,260000.0,USD,4587570.00,260000.00,160.0,140.0,1625.000000,1857.142857,NaN,NaN,ESPECTACULAR PH CON COCHERA CUBIERTA !! 160 m2...


<h3> Completamos datos faltantes de superficie </h3>

[volver a TOC](#section_toc)

In [25]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba_clean.surface_covered_in_m2.fillna(data_caba_clean.surface_total_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
53225,apartment,Belgrano,600000.0,USD,10586700.00,600000.0,155.0,155.0,NaN,3870.967742,NaN,4.0,"Venta - Departamento - Argentina, Buenos Aires..."
15511,apartment,Palermo Chico,640000.0,USD,11292480.00,640000.0,201.0,188.0,3184.079602,3404.255319,NaN,NaN,"Piso en el corazon de Palermo Chico, gran plan..."
88705,apartment,Boca,44000.0,USD,776358.00,44000.0,23.0,23.0,1913.043478,1913.043478,NaN,NaN,Sobre la Avenida Almirante Brown: A Pasos de N...
33196,apartment,Colegiales,120000.0,USD,2117340.00,120000.0,42.0,42.0,2857.142857,2857.142857,NaN,1.0,"Ravignani 1453, CABA / departamento Monoambien..."
78714,apartment,Flores,85500.0,USD,1508604.75,85500.0,54.0,49.0,1583.333333,1744.897959,NaN,NaN,Venta Departamento 2 Ambientes en Pozo – Flore...
34196,apartment,Caballito,179000.0,USD,3158365.50,179000.0,70.0,70.0,2557.142857,2557.142857,NaN,NaN,CODIGO: ubicado en: alberdi 1500 - Publicado...
76043,apartment,Caballito,195000.0,USD,3440677.50,195000.0,94.0,82.0,2074.468085,2378.048780,NaN,NaN,Departamento en duplex de 2 dormitorios en Cab...
112942,PH,Boedo,145000.0,USD,2558452.50,145000.0,120.0,90.0,1208.333333,1611.111111,NaN,NaN,CODIGO: 2883-886 ubicado en: Las Casas - Pub...
111786,apartment,Monserrat,103400.0,USD,1824441.30,103400.0,53.0,49.0,1950.943396,2110.204082,NaN,NaN,Departamentos en venta con piso flotante mela...
24961,apartment,Belgrano,95000.0,USD,1676227.50,95000.0,90.0,48.0,1055.555556,1979.166667,NaN,2.0,EXCELENTE DEPARTAMENTO DE 2 AMBIENTES CON TERR...


<h3> Completamos datos faltantes de price_usd_per_m2 </h3>

[volver a TOC](#section_toc)

In [26]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba_clean.price_usd_per_m2.fillna(data_caba_clean.price_aprox_usd / data_caba_clean.surface_total_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
81409,apartment,Belgrano,1200000.0,USD,21173400.0,1200000.0,170.0,150.0,7058.823529,8000.000000,NaN,NaN,Quartier Libertador. Torre de Lujo!. Piso alto...
61211,apartment,Villa Crespo,165000.0,USD,2911342.5,165000.0,81.0,64.0,2037.037037,2578.125000,NaN,3.0,Departamento 3 AMBIENTES en Villa CrespoFrente...
15279,house,Floresta,250000.0,USD,4411125.0,250000.0,160.0,144.0,1562.500000,1736.111111,NaN,NaN,CODIGO: ubicado en: Murature al 4800 - Publi...
33780,apartment,Villa Pueyrredón,92000.0,USD,1623294.0,92000.0,40.0,40.0,2300.000000,2300.000000,NaN,1.0,Corredor Responsable: Gustavo Guastello - C.U....
49633,apartment,Almagro,125000.0,USD,2205562.5,125000.0,60.0,44.0,2083.333333,2840.909091,NaN,2.0,"Sup total: 60,45 m2VENTA DIRECTA, el propietar..."
14757,apartment,San Telmo,155000.0,USD,2734897.5,155000.0,77.0,73.0,2012.987013,2123.287671,NaN,3.0,El inmueble esta muy bien ubicado en el Casco ...
7148,apartment,Palermo,810000.0,USD,14292045.0,810000.0,210.0,195.0,3857.142857,4153.846154,NaN,5.0,Excelente duplex ubicado en una de las mejores...
44918,apartment,Villa Crespo,219000.0,USD,3864145.5,219000.0,100.0,94.0,2190.000000,2329.787234,NaN,4.0,VENDO DPTO 4 AMB Va. Crespo a 1 Cuadra Subte4 ...
114445,apartment,Recoleta,279000.0,USD,4922815.5,279000.0,90.0,90.0,3100.000000,3100.000000,NaN,3.0,VENTA Depto. de 3 Ambientes C/DEPENDENCIA Fre...
30526,apartment,Villa Santa Rita,70200.0,USD,1238643.9,70200.0,39.0,39.0,1800.000000,1800.000000,NaN,1.0,Corredor Responsable: Ariel Champanier - C.U.C...


<h3> Eliminamos floor </h3>

[volver a TOC](#section_toc)

In [33]:
display(data_caba.floor.isnull().count())

# El piso es un dato muy incompleto por lo que se dropea

# Dropeamos la columna creada anteriormente.
data_caba_clean.drop(columns=['floor'], inplace=True)

# En el caso de Rooms lo reemplazamos por 0 --> Lo pasé más abajo
# data_caba_clean.rooms.fillna(0, inplace=True)

32316

KeyError: "['floor'] not found in axis"

In [38]:
display(data_caba.rooms.count())
# En el caso de rooms, podemos tomar que el dato para los NaN es 0?? modifica esto el analisis para predecir? 

15325

<h3> Eliminamos columnas de precios en moneda local </h3>

[volver a TOC](#section_toc)

In [29]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba_clean.drop(columns=['price', 'currency','price_per_m2'], inplace=True)

data_caba_clean.sample(20)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,description
85269,apartment,Coghlan,3617122.50,205000.0,115.0,98.0,1782.608696,NaN,* Los valores de superficies y expensas son ap...
86533,apartment,Palermo Hollywood,7410690.00,420000.0,140.0,140.0,3000.000000,NaN,"Hermoso departamento, con luz todo el día, coc..."
45809,apartment,Balvanera,2434941.00,138000.0,82.0,82.0,1682.926829,3.0,Corredor Responsable: Ariel Champanier - C.U.C...
30263,apartment,Belgrano,4746370.50,269000.0,110.0,105.0,2445.454545,4.0,"Departamento en el barrio de Belgrano, sobre l..."
94127,apartment,Puerto Madero,13127508.00,744000.0,115.0,82.0,6469.565217,NaN,Selectas residencias exclusivas con gran canti...
89677,apartment,San Telmo,1242172.80,70400.0,38.0,32.0,1852.631579,NaN,Bernardo de Irigoyen 1418: Ubicada en pleno co...
43340,apartment,Nuñez,2468465.55,139900.0,53.0,53.0,2639.622642,2.0,Corredor Responsable: Jorge Salafia - CUCICBA ...
33944,PH,Pompeya,2029117.50,115000.0,66.0,66.0,1742.424242,4.0,Corredor Responsable: Ariel Champanier - C.U.C...
77621,apartment,San Telmo,2734897.50,155000.0,85.0,80.0,1823.529412,NaN,3 ambientes al frente con balcón corrido. SE V...
79953,apartment,Belgrano,3306579.30,187400.0,72.0,65.0,2602.777778,NaN,PRE-VENTA DEPARTAMENTO DE 3 AMBIENTES Y 2 BALC...


<h3> Nuevo recuento de nulos </h3>

In [30]:
print('\nPorcentaje de valores incompletos por columna:')
print(f'{round(100 * data_caba_clean.isnull().sum()/data_caba_clean.shape[0], 2)}')

display(data_caba_clean.shape)


Porcentaje de valores incompletos por columna:
property_type                  0.00
place_name                     0.00
price_aprox_local_currency     0.00
price_aprox_usd                0.00
surface_total_in_m2            0.00
surface_covered_in_m2          0.00
price_usd_per_m2               0.00
rooms                         43.82
description                    0.00
dtype: float64


(25627, 9)

<h3>Creación de una columna para categorizar por metros cuadrados

In [31]:
# Delimitamos los bins para realizar un cut
# bins = [20, 30, 45, 75, 150, 220]
bins = [0, 30, 45, 75, 150, 220]
labels = ['mono', 's45', 's75', 's150', 'm220' ]

data_caba_clean['m2_categories'] = pd.cut(data_caba_clean.surface_covered_in_m2, bins, labels)
data_caba_clean['m2_labels'] = pd.cut(x=data_caba_clean.surface_covered_in_m2, bins=bins, labels=labels, right=False)


In [32]:
data_caba_clean.m2_categories.value_counts().sort_index()

(0, 30]       2530
(30, 45]      7452
(45, 75]      7227
(75, 150]     5449
(150, 220]    1572
Name: m2_categories, dtype: int64

In [33]:
data_caba_clean.m2_labels.value_counts().sort_index()

mono    1633
s45     7863
s75     7475
s150    5574
m220    1628
Name: m2_labels, dtype: int64

<a id="section_caba_description"></a> 
<h3> Análisis del campo <strong>description</strong></h3>

[volver a TOC](#section_toc)

Normalización del campo, todo minuscula, sin tildes, etc

In [34]:
data_caba_clean.description = data_caba_clean.description.str.lower()

data_caba_clean.description.sample(10)

88442     forma de pago: 30% en la firma de la adhesión ...
54530     ubicado a mts de avenida cramer, 5 cuadras de ...
12730     dpto a estrenar de 2 amb con amplias dimension...
51692     planta baja de 33.29 m2 + entrepiso de 19.57 m...
53742     corredor responsable: hernan perrone - c.u.c.i...
116608    monoambiente/estudio al contrafrente apto prof...
79186     esplendido duplex en venta en las cañitas de 2...
52200     corredor responsable: rebeca stern - c.u.c.i.c...
67657     hermoso ambiente al frente!  kitchenette, bano...
914       venta de departamento 2 ambientes con cochera ...
Name: description, dtype: object

<a id="section_caba_description_amb"></a> 
<h4> Cantidad de ambientes</h4>

[volver a TOC](#section_toc)

In [35]:
amb_pattern = "((?P<numero_amb>\d)(\s)?(amb|anv|amv|anb))"
amb_pattern_regex = re.compile(amb_pattern)

In [36]:
amb_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb_pattern_regex.search(x))
mask_amb_match_notnull = amb_match.notnull()

# Aplicamos el grupo que encuentra el dígito de cantidad de ambientes, casteado como integer
data_caba_clean.loc[mask_amb_match_notnull, 'number_rooms'] = \
amb_match.loc[mask_amb_match_notnull].apply(lambda x: int(x.group("numero_amb")))

In [37]:
dorm_pattern = "((?P<numero_dorm>\d)(\s)?(dorm))"
dorm_pattern_regex = re.compile(dorm_pattern)

In [38]:
dorm_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm_pattern_regex.search(x))

mask_dorm_match_notnull = dorm_match.notnull()

data_caba_clean.loc[mask_dorm_match_notnull, 'number_rooms'] = \
dorm_match.loc[mask_dorm_match_notnull].apply(lambda x: int(x.group("numero_dorm"))+1)

#se asume que el numero de ambientes es numero_dorm +1

Tratamos de llenar los rooms que faltan con lo que obtuvimos de la descripción

In [39]:
data_caba_clean.head(3)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,description,m2_categories,m2_labels,number_rooms
0,PH,Mataderos,1093959.0,62000.0,55.0,40.0,1127.272727,NaN,"2 ambientes tipo casa planta baja por pasillo,...","(30, 45]",s45,2.0
2,apartment,Mataderos,1270404.0,72000.0,55.0,55.0,1309.090909,NaN,2 ambientes 3er piso lateral living comedor am...,"(45, 75]",s75,2.0
7,apartment,Belgrano,2434941.0,138000.0,45.0,40.0,3066.666667,NaN,excelente monoambiente a estrenar amplio super...,"(30, 45]",s45,NaN


In [40]:
data_caba_clean.rooms.isnull().sum()

11230

In [41]:
data_caba_clean.number_rooms.isnull().sum()

9881

In [42]:
data_caba_clean.rooms.fillna(data_caba_clean.number_rooms, inplace=True)

In [43]:
data_caba_clean.rooms.isnull().sum()

4665

In [44]:
# data_caba_clean.rooms.fillna(0, inplace=True)

In [45]:
data_caba_clean.rooms.value_counts()

2.0     5406
3.0     5262
1.0     4341
4.0     4012
5.0     1266
6.0      362
7.0      174
8.0       57
9.0       27
10.0      24
12.0      11
11.0       9
0.0        4
17.0       3
22.0       1
15.0       1
14.0       1
13.0       1
Name: rooms, dtype: int64

### Llenamos los que faltan según las categorías de los bins???
Además: eliminar los que tengan más de 10 ambientes? Son pocos datos! 

<a id="section_caba_description_amb"></a> 
<h4> Amenities</h4>

[volver a TOC](#section_toc)

<h5> Pileta </h5>

In [46]:
pileta_pattern = "(?P<pileta>pileta|piscina|picina|pisina|pool)"
pileta_pattern_regex = re.compile(pileta_pattern)

In [47]:
pileta_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      pileta_pattern_regex.search(x))
mask_pileta_match_notnull = pileta_match.notnull()

data_caba_clean.loc[mask_pileta_match_notnull, 'pool'] = 1

In [48]:
# data_caba_clean['pool']

In [49]:
data_caba_clean['pool'].notnull().sum()

6161

In [50]:
#Completamos los datos NaN con ceros (convertimos en una variable dummy)
data_caba_clean.pool.fillna(0, inplace = True)

<h5> Laundry </h5>

In [51]:
laudry_pattern = "(?P<laundry>laundry|lavadero)"
laundry_pattern_regex = re.compile(laudry_pattern)

laundry_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      laundry_pattern_regex.search(x))
mask_laundry_match_notnull = laundry_match.notnull()

data_caba_clean.loc[mask_laundry_match_notnull, 'laundry'] = 1

In [52]:
data_caba_clean['laundry'].notnull().sum()

10958

In [53]:
data_caba_clean.laundry.fillna(0, inplace = True)

<h5> Parking </h5>

In [54]:
parking_pattern = "(?P<parking>parking|estacionamiento|garage|cochera|garaje)"
parking_pattern_regex = re.compile(parking_pattern)

parking_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      parking_pattern_regex.search(x))
mask_parking_match_notnull = parking_match.notnull()

data_caba_clean.loc[mask_parking_match_notnull, 'parking'] = 1

In [55]:
data_caba_clean['parking'].notnull().sum()

9796

In [56]:
data_caba_clean.parking.fillna(0, inplace = True)

<h5> Gimnasio </h5>

In [57]:
gym_pattern = "(?P<gimnasio>gim|gym|fitness|fitnes|ejercicio)"
gym_pattern_regex = re.compile(gym_pattern)

gym_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      gym_pattern_regex.search(x))
mask_gim_match_notnull = gym_match.notnull()

data_caba_clean.loc[mask_gim_match_notnull, 'gym'] = 1

In [58]:
data_caba_clean['gym'].notnull().sum()

3363

In [59]:
data_caba_clean.gym.fillna(0, inplace = True)

<a id="section_caba_dummies"></a> 
<h3> Creación dummies sobre datos categóricos </h3>

[volver a TOC](#section_toc)

In [60]:
# Usamos la función get_dummies con one-hot encoding (drop_first=True)
property_type_dummies = pd.get_dummies(data_caba_clean['property_type'], drop_first = True, prefix='prop_type')
property_type_dummies

,prop_type_apartment,prop_type_house,prop_type_store
0,0,0,0
2,1,0,0
7,1,0,0
8,1,0,0
13,1,0,0
...,...,...,...
121153,1,0,0
121157,1,0,0
121158,1,0,0
121215,1,0,0


In [61]:
m2_labels_dummies = pd.get_dummies(data_caba_clean['m2_labels'], drop_first = True, prefix='m2_label')

In [62]:
place_name_dummies = pd.get_dummies(data_caba_clean['place_name'], drop_first = True, prefix='place_name')

In [ ]:
# Hacemos un join para concatenar las columnas al dataframe
data_caba_clean.join([property_type_dummies,m2_labels_dummies, place_name_dummies])

In [ ]:
data_caba_clean.head(3)

### Falta reindexar las filas.

### Falta el dummy de Rooms pero primero hay que ver si completamos los datos con la info de los bins

In [64]:
# Exportamos el archivo para usar en otras notebooks, esto lo guarda en el mismo directorio en el que estamos operando
# data_caba_clean.to_csv('data_caba_clean.csv')